In [3]:
import numpy as np
import pandas as pd

In [4]:
df_movies = pd.read_csv('datasets/MovieLens/movies.csv')

print("Archivo cargado correctamente")

Archivo cargado correctamente


In [5]:
df_ratings = pd.read_csv('datasets/MovieLens/ratings.csv')

print("Archivo cargado correctamente")

Archivo cargado correctamente


In [6]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [8]:
df_prom_ratings = df_ratings.groupby('movieId')['rating'].mean().reset_index()

df_prom_ratings.sort_values(by='rating', ascending=False, axis=0, inplace=True)
df_prom_ratings.head(100)


,movieId,rating
29523,136782,5.0
49654,186119,5.0
29643,137032,5.0
49041,184643,5.0
29646,137038,5.0
...,...,...
29914,137731,5.0
29894,137668,5.0
49323,185305,5.0
29766,137329,5.0


In [9]:
merged_df = df_movies.merge(df_prom_ratings, on='movieId')
merged_df.sort_values(by='rating', ascending=False, axis=0, inplace=True)
merged_df.head()

,movieId,title,genres,rating
29523,136782,The Girl is in Trouble (2015),Thriller,5.0
49654,186119,A Gift Horse (2015),Children,5.0
29643,137032,The Perfect Neighbor (2005),Drama|Thriller,5.0
49041,184643,Relentless (2018),Thriller,5.0
29646,137038,The Perfect Wife (2001),Drama|Thriller,5.0


In [10]:
num_votos = df_ratings['movieId'].value_counts().reset_index()
merged_final_df = merged_df.merge(num_votos, on='movieId')
merged_final_df.sort_values(by='count', ascending=False, axis=0, inplace=True)
merged_final_df

,movieId,title,genres,rating,count
2566,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.048011,81491
1532,318,"Shawshank Redemption, The (1994)",Crime|Drama,4.413576,81482
2038,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.188912,79672
2209,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.151342,74127
2201,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.154099,72674
...,...,...,...,...,...
35504,138484,Just a Father (2008),(no genres listed),3.000000,1
35505,196833,Cradle to Grave,Documentary,3.000000,1
35507,196715,Two Balloons (2017),Animation,3.000000,1
35508,196709,Welcome 2 Karachi (2015),Comedy,3.000000,1


In [11]:
df_filtrado = merged_final_df.loc[merged_final_df['count'] >= 6000]
df_filtrado.sort_values(by='rating', ascending=False, axis=0, inplace=True)

df_filtrado.drop(['genres'], axis=1, inplace=True)



df_filtrado

C:\Users\mvill\AppData\Local\Temp\ipykernel_3348\4258900442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.sort_values(by='rating', ascending=False, axis=0, inplace=True)
C:\Users\mvill\AppData\Local\Temp\ipykernel_3348\4258900442.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.drop(['genres'], axis=1, inplace=True)


,movieId,title,rating,count
1532,318,"Shawshank Redemption, The (1994)",4.413576,81482
1633,858,"Godfather, The (1972)",4.324336,52498
1648,50,"Usual Suspects, The (1995)",4.284353,55366
1652,1221,"Godfather: Part II, The (1974)",4.261759,34188
1655,2019,Seven Samurai (Shichinin no samurai) (1954),4.254769,13367
...,...,...,...,...
50306,63992,Twilight (2008),2.391415,6115
50572,1882,Godzilla (1998),2.373145,8017
51144,2701,Wild Wild West (1999),2.319671,11723
52240,193,Showgirls (1995),2.247830,7142


In [12]:
df_movies_meta = pd.read_csv('datasets/IMBD/movies_metadata.csv')

df_movies_meta = df_movies_meta[['id', 'title', 'budget', 'revenue']]

df_movies_meta.head()

C:\Users\mvill\AppData\Local\Temp\ipykernel_3348\4103628692.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_meta = pd.read_csv('datasets/IMBD/movies_metadata.csv')


,id,title,budget,revenue
0,862,Toy Story,30000000,373554033.0
1,8844,Jumanji,65000000,262797249.0
2,15602,Grumpier Old Men,0,0.0
3,31357,Waiting to Exhale,16000000,81452156.0
4,11862,Father of the Bride Part II,0,76578911.0


In [15]:
df_links = pd.read_csv('datasets/MovieLens/links.csv')

df_links = df_links[['movieId', 'tmdbId']]

# Cambia el nombre del a segunda columna para que coincida con el df_movies_meta
df_links.rename(columns={'tmdbId': 'id'}, inplace=True)

# Convierte el id a entero
pd.to_numeric(df_links['id'], errors='coerce', downcast='integer')


df_links.head()



,movieId,id
0,1,862.0
1,2,8844.0
2,3,15602.0
3,4,31357.0
4,5,11862.0


In [16]:
# Concatena los df de links y movies_meta usando Concat

df_movies_meta = pd.concat([df_movies_meta, df_links], axis=1)

In [17]:
df_movies_meta = df_movies_meta[['movieId', 'budget', 'revenue']]

df_movies_meta.head()

,movieId,budget,revenue
0,1,30000000,373554033.0
1,2,65000000,262797249.0
2,3,0,0.0
3,4,16000000,81452156.0
4,5,0,76578911.0


In [18]:
# Une el df filtrado y el df de movies_meta usando merge

df_final = df_filtrado.merge(df_movies_meta, on='movieId')

In [19]:
df_final.head()

,movieId,title,rating,count,budget,revenue
0,318,"Shawshank Redemption, The (1994)",4.413576,81482,25000000,28341469.0
1,858,"Godfather, The (1972)",4.324336,52498,4500000,7011317.0
2,50,"Usual Suspects, The (1995)",4.284353,55366,6000000,23341568.0
3,1221,"Godfather: Part II, The (1974)",4.261759,34188,5500000,159616327.0
4,2019,Seven Samurai (Shichinin no samurai) (1954),4.254769,13367,5000000,8153677.0
